# 青空文庫データコンバーター

青空文庫の .txt 形式のファイルを適当なディレクトリに複数置いておけば、それらをつなげて本文の文（。で区切られる）毎に一行となるフォーマットに変換してくれます。
雑にやってるので章や物語の切れ目などが考えられていなかったり、発話の途中の。で切られてたりします。

In [ ]:
# カレントディレクトリをリポジトリ直下にするおまじない
import os
while os.getcwd().split('/')[-1] != 'deep_dialog_tutorial': os.chdir('..')
print('current dir:', os.getcwd())

In [ ]:
import os
import re
import codecs
from tqdm import tqdm

In [ ]:
# ここに青空文庫からダウンロードしたファイルを配置してください
data_dir = 'tmp/natsume/'
out_path = 'data/natsume.txt'

os.makedirs(data_dir, exist_ok=True)

In [ ]:
def convert_line(line):
    line = line.replace('\r\n', '').replace('○', '').replace('\u3000', '').replace('※', '').replace('｜', '')
    line = re.sub(r'《.*?》', '', line)
    line = re.sub(r'\［.*?\］', '', line)
    if not line:
        return []
    sentence_list = line.split('。')
    return [sentence + '。' for sentence in sentence_list if sentence]

def get_text_list(file_path):
    with codecs.open(file_path, mode='r', encoding='shift_jis') as f:
        header_sep_count = 0
        text_list = []
        for line in f.readlines():
            if line.startswith('-----------------'):
                header_sep_count += 1
                continue
            if header_sep_count < 2:
                continue
            if line.startswith('底本：'):
                break
            text_list += convert_line(line)
        return text_list

In [ ]:
text_list = []

file_name_list = os.listdir(data_dir)
for file_name in tqdm(file_name_list):
    file_path = os.path.join(data_dir, file_name)
    text_list += get_text_list(file_path)
print('{} lines'.format(len(text_list)))

In [ ]:
with codecs.open(out_path, mode='w', encoding='utf-8') as f:
    f.write('\n'.join(text_list) + '\n')